<a href="https://colab.research.google.com/github/kurtsenol/machine-learning/blob/master/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
DATA_PATH = 'drive/My Drive/nipstxt/'

In [38]:
folders = ["nips{0:02}".format(i) for i in range(0,13)]
folders

['nips00',
 'nips01',
 'nips02',
 'nips03',
 'nips04',
 'nips05',
 'nips06',
 'nips07',
 'nips08',
 'nips09',
 'nips10',
 'nips11',
 'nips12']

In [12]:
# # Read all texts into a list.
# papers = []

# for folder in folders:
#   file_names = os.listdir(DATA_PATH + folder)
#   for file_name in file_names:
#     with open(DATA_PATH + folder + '/' + file_name, encoding='utf-8', errors='ignore', mode='r+') as f:
#       data = f.read()
#     papers.append(data)
# len(papers)

1740

In [0]:
# import pickle

# with open("drive/My Drive/nipstxt/papers_pickle.txt", "wb") as fp:   #Pickling
#   pickle.dump(papers, fp)

In [0]:
with open("drive/My Drive/nipstxt/papers_pickle.txt", "rb") as fp:   # Unpickling
  papers = pickle.load(fp)

In [41]:
len(papers)

1740

In [42]:
print(papers[0][:1000])

1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a problem 
from examples using a local learning rule, we prove that the entropy of the 
problem becomes a lower bound for the connectivity of the network. 
INTRODUCTION 
The most distinguishing feature of neural networks is their ability to spon- 
taneously learn the desired function from 'training' samples, i.e., their ability 
to program themselves. Clearly, a given neural network cannot just learn any 
function, there must be some restrictions on which networks can learn which 
functions. One obv

In [43]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

In [0]:
def normalize_corpus(papers):
  norm_papers = []
  for paper in papers:
    paper = paper.lower()
    paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
    paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
    paper_tokens = [token for token in paper_tokens if len(token) > 1]
    paper_tokens = [token for token in paper_tokens if token not in stop_words]
    
    paper_tokens = list(filter(None, paper_tokens))
    if paper_tokens:
      norm_papers.append(paper_tokens)
  return norm_papers

In [46]:
norm_papers = normalize_corpus(papers)
print(len(norm_papers))

1740


In [47]:
# viewing a processed paper
print(norm_papers[0][:50])

['connectivity', 'versus', 'entropy', 'yaser', 'abu', 'mostafa', 'california', 'institute', 'technology', 'pasadena', 'ca', 'abstract', 'doe', 'connectivity', 'neural', 'network', 'number', 'synapsis', 'per', 'neuron', 'relate', 'complexity', 'problem', 'handle', 'measured', 'entropy', 'switching', 'theory', 'would', 'suggest', 'relation', 'since', 'boolean', 'function', 'implemented', 'using', 'circuit', 'low', 'connectivity', 'using', 'two', 'input', 'nand', 'gate', 'however', 'network', 'learns', 'problem', 'example', 'using']


In [0]:
import gensim


In [0]:
bigram = gensim.models.Phrases(norm_papers, min_count=20, threshold=20, delimiter=b'_') # higher threshold fewer phrases.
bigram_model = gensim.models.phrases.Phraser(bigram)

In [50]:
# sample demonstration
print(bigram_model[norm_papers[0]][:50])

['connectivity', 'versus', 'entropy', 'yaser', 'abu_mostafa', 'california_institute', 'technology_pasadena', 'ca_abstract', 'doe', 'connectivity', 'neural_network', 'number', 'synapsis', 'per', 'neuron', 'relate', 'complexity', 'problem', 'handle', 'measured', 'entropy', 'switching', 'theory', 'would', 'suggest', 'relation', 'since', 'boolean_function', 'implemented', 'using', 'circuit', 'low', 'connectivity', 'using', 'two', 'input', 'nand', 'gate', 'however', 'network', 'learns', 'problem', 'example', 'using', 'local', 'learning', 'rule', 'prove', 'entropy', 'problem']


Let’s generate phrases for all our tokenized research papers and build a vocabulary that will help us obtain a unique term/phrase to number mapping (since machine or deep learning only works on numeric tensors).

In [0]:
norm_corpus_bigrams = [bigram_model[doc] for doc in norm_papers]

In [52]:
# Create a dictionary representation of the documents.
dictionary = gensim.corpora.Dictionary(norm_corpus_bigrams)

print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, '0a'), (1, '2h'), (2, '2h2'), (3, '2he'), (4, '2n'), (5, '__c'), (6, '_c'), (7, '_k'), (8, 'a2'), (9, 'ability'), (10, 'abu_mostafa'), (11, 'access'), (12, 'accommodate'), (13, 'according'), (14, 'accumulated')]
Total Vocabulary Size: 78892


Several of these terms are not very useful since they are specific to a paper or even a paragraph in a research paper. Hence, it is time to prune
our vocabulary and start removing terms. Leveraging document frequency is a great way to achieve this. By now, you probably realize that the document frequency of a term is basically the total number of times that term occurs across all the documents in a corpus.

In [53]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.

dictionary.filter_extremes(no_below=20, no_above=0.6)

print('Total Vocabulary Size:', len(dictionary))

Total Vocabulary Size: 7756


In [54]:
print('Sample word to number mappings:', list(dictionary.items())[:15])

Sample word to number mappings: [(0, '2n'), (1, '_c'), (2, 'a2'), (3, 'ability'), (4, 'abu_mostafa'), (5, 'access'), (6, 'accommodate'), (7, 'according'), (8, 'accumulated'), (9, 'acknowledgement_work'), (10, 'addison_wesley'), (11, 'afosr'), (12, 'aip'), (13, 'air_force'), (14, 'although')]


We removed all terms that occur fewer than 20 times across all documents and all
terms that occur in more than 60% of all the documents. We are interested in finding different themes and topics and not recurring themes. Hence, this suits our scenario perfectly. We can now perform feature engineering by leveraging a simple Bag of Words model.

In [55]:
# Transforming corpus into bag of words vectors

bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigrams]
print(bow_corpus[1][:50])

[(3, 2), (4, 1), (11, 1), (13, 1), (15, 1), (21, 1), (27, 1), (30, 1), (33, 1), (35, 2), (36, 1), (45, 1), (46, 1), (47, 1), (48, 2), (57, 2), (64, 7), (67, 4), (71, 3), (78, 1), (92, 3), (94, 2), (97, 9), (104, 1), (113, 1), (115, 4), (118, 4), (120, 2), (124, 4), (125, 3), (128, 1), (133, 5), (136, 3), (138, 1), (142, 3), (144, 6), (152, 2), (155, 1), (156, 1), (158, 1), (162, 1), (168, 1), (172, 1), (177, 12), (199, 1), (203, 14), (204, 1), (213, 1), (215, 1), (218, 4)]


In [56]:
# viewing actual terms and their counts
print([(dictionary[idx] , freq) for idx, freq in bow_corpus[1][:50]])

[('ability', 2), ('abu_mostafa', 1), ('afosr', 1), ('air_force', 1), ('american_institute', 1), ('appendix', 1), ('assume', 1), ('asymptotic', 1), ('axe', 1), ('become', 2), ('becomes', 1), ('bt', 1), ('ca_abstract', 1), ('california_institute', 1), ('cannot', 2), ('complete', 2), ('connected', 7), ('consider', 4), ('corresponding', 3), ('denote', 1), ('ea', 3), ('ed', 2), ('element', 9), ('environment', 1), ('expected', 1), ('expression', 4), ('fact', 4), ('final', 2), ('fixed', 4), ('follows', 3), ('furthermore', 1), ('get', 5), ('go', 3), ('going', 1), ('hand', 3), ('hence', 6), ('implemented', 2), ('independent', 1), ('independently', 1), ('interested', 1), ('ity', 1), ('know', 1), ('le', 1), ('let', 12), ('need', 1), ('neuron', 14), ('next_section', 1), ('occur', 1), ('office_scientific', 1), ('otherwise', 4)]


In [57]:
# total papers in the corpus

print('Total number of papers:', len(bow_corpus))

Total number of papers: 1740


### Latent Dirichlet Allocation

In [0]:
TOTAL_TOPICS = 10

In [0]:
lda_model = gensim.models.LdaModel(corpus=bow_corpus, id2word=dictionary, chunksize=1740, alpha='auto',
                                   eta='auto', random_state=42, iterations=500, num_topics=TOTAL_TOPICS,
                                   passes=20, eval_every=None)

Viewing the topics in our trained topic model is quite easy and we can generate them with the following code.

In [61]:
for topic_id, topic in lda_model.print_topics(num_topics=10, num_words=20):
  print('Topic #'+str(topic_id+1)+':')
  print(topic)
  print()

Topic #1:
0.014*"control" + 0.008*"unit" + 0.007*"state" + 0.007*"trajectory" + 0.006*"target" + 0.006*"dynamic" + 0.006*"movement" + 0.006*"motor" + 0.005*"position" + 0.005*"module" + 0.005*"task" + 0.005*"controller" + 0.005*"representation" + 0.004*"pattern" + 0.004*"change" + 0.004*"subject" + 0.004*"hand" + 0.004*"behavior" + 0.004*"arm" + 0.003*"activation"

Topic #2:
0.012*"chip" + 0.011*"circuit" + 0.008*"image" + 0.007*"current" + 0.007*"neuron" + 0.006*"voltage" + 0.006*"analog" + 0.004*"processor" + 0.004*"signal" + 0.004*"implementation" + 0.004*"bit" + 0.004*"design" + 0.004*"pixel" + 0.004*"line" + 0.003*"vector" + 0.003*"neural" + 0.003*"device" + 0.003*"digital" + 0.003*"application" + 0.003*"architecture"

Topic #3:
0.027*"state" + 0.014*"action" + 0.009*"policy" + 0.008*"control" + 0.007*"step" + 0.007*"reinforcement_learning" + 0.006*"optimal" + 0.005*"task" + 0.005*"environment" + 0.004*"reward" + 0.004*"goal" + 0.004*"agent" + 0.004*"td" + 0.004*"cost" + 0.004*"ro

all the weights are the same sign and tell us the importance of each term in the topic. We can also view the overall mean coherence score of the model.

In [62]:
topics_coherences = lda_model.top_topics(bow_corpus, topn=20)

avg_coherence_score = np.mean([item[1] for item in topics_coherences])

print('Avg. Coherence Score:', avg_coherence_score)

Avg. Coherence Score: -1.0190235012946323


Let’s now look at the output of our LDA topic model in an easier to understand format. One way is to visualize the topics as tuples of terms and weights.

In [64]:
topics_with_wts = [item[0] for item in topics_coherences]

print('LDA Topics with Weights')
print('='*50)

for idx, topic in enumerate(topics_with_wts):
  print('Topic #'+str(idx+1)+':')
  print([(term, round(wt, 3)) for wt, term in topic])
  print()

LDA Topics with Weights
Topic #1:
[('neuron', 0.02), ('cell', 0.017), ('response', 0.01), ('stimulus', 0.008), ('activity', 0.007), ('signal', 0.007), ('spike', 0.006), ('pattern', 0.006), ('synaptic', 0.005), ('frequency', 0.005), ('neural', 0.005), ('effect', 0.004), ('firing', 0.004), ('cortical', 0.004), ('noise', 0.004), ('connection', 0.004), ('et_al', 0.003), ('temporal', 0.003), ('change', 0.003), ('current', 0.003)]

Topic #2:
[('distribution', 0.009), ('probability', 0.007), ('vector', 0.006), ('matrix', 0.005), ('approximation', 0.005), ('class', 0.005), ('variable', 0.004), ('estimate', 0.004), ('density', 0.004), ('sample', 0.004), ('linear', 0.004), ('gaussian', 0.004), ('equation', 0.003), ('component', 0.003), ('let', 0.003), ('solution', 0.003), ('bound', 0.003), ('theory', 0.003), ('optimal', 0.003), ('xi', 0.003)]

Topic #3:
[('state', 0.009), ('vector', 0.008), ('unit', 0.008), ('node', 0.007), ('neuron', 0.007), ('memory', 0.005), ('dynamic', 0.005), ('pattern', 0.

We can also view the topics as a list of terms without the weights when we want to understand the context or theme conveyed by each topic.

In [65]:
print('LDA Topics without Weights')
print('='*50)

for idx, topic in enumerate(topics_with_wts):
  print('Topic #'+str(idx+1)+':')
  print([term for wt, term in topic])
  print()

LDA Topics without Weights
Topic #1:
['neuron', 'cell', 'response', 'stimulus', 'activity', 'signal', 'spike', 'pattern', 'synaptic', 'frequency', 'neural', 'effect', 'firing', 'cortical', 'noise', 'connection', 'et_al', 'temporal', 'change', 'current']

Topic #2:
['distribution', 'probability', 'vector', 'matrix', 'approximation', 'class', 'variable', 'estimate', 'density', 'sample', 'linear', 'gaussian', 'equation', 'component', 'let', 'solution', 'bound', 'theory', 'optimal', 'xi']

Topic #3:
['state', 'vector', 'unit', 'node', 'neuron', 'memory', 'dynamic', 'pattern', 'matrix', 'equation', 'net', 'sequence', 'layer', 'threshold', 'activation', 'linear', 'size', 'let', 'recurrent', 'rule']

Topic #4:
['chip', 'circuit', 'image', 'current', 'neuron', 'voltage', 'analog', 'processor', 'signal', 'implementation', 'bit', 'design', 'pixel', 'line', 'vector', 'neural', 'device', 'digital', 'application', 'architecture']

Topic #5:
['training', 'prediction', 'distribution', 'variable', 'no

We can use perplexity and coherence scores as measures to evaluate the topic
model. Typically, lower the perplexity, the better the model. Similarly, the lower the UMass score and the higher the Cv score in coherence, the better the model.

In [0]:
cv_coherence_model_lda = gensim.models.CoherenceModel( model=lda_model, corpus=bow_corpus, texts=norm_corpus_bigrams,
                                                      dictionary=dictionary, coherence='c_v')

avg_coherence_cv = cv_coherence_model_lda.get_coherence()

umass_coherence_model_lda = gensim.models.CoherenceModel( model=lda_model, corpus=bow_corpus, texts=norm_corpus_bigrams,
                                                          dictionary=dictionary, coherence='u_mass')


In [67]:

avg_coherence_umass = umass_coherence_model_lda.get_coherence()
perplexity = lda_model.log_perplexity(bow_corpus)

print('Avg. Coherence Score (Cv):', avg_coherence_cv)
print('Avg. Coherence Score (UMass):', avg_coherence_umass)
print('Model Perplexity:', perplexity)

Avg. Coherence Score (Cv): 0.4952768212161648
Avg. Coherence Score (UMass): -1.019023501294632
Model Perplexity: -7.786467030429068


### LDA Models with MALLET

In [69]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2020-05-22 21:37:12--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  11.0MB/s    in 1.4s    

2020-05-22 21:37:14 (11.0 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [75]:
!unzip mallet-2.0.8.zip

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
MALLET_PATH = '/content/mallet-2.0.8/bin/mallet'

In [77]:
lda_mallet = gensim.models.wrappers.LdaMallet( mallet_path=MALLET_PATH, corpus=bow_corpus,
                                              num_topics=TOTAL_TOPICS, id2word=dictionary,
                                               iterations=500, workers=16)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


We can now look at the generated topics by leveraging the following code snippet.

In [0]:
topics = [[(term, round(wt, 3)) for term, wt in lda_mallet.show_topic(n, topn=20)] for n in range(0, TOTAL_TOPICS)]


In [79]:
for idx, topic in enumerate(topics):
  print('Topic #'+str(idx+1)+':')
  print([term for term, wt in topic])
  print()

Topic #1:
['circuit', 'signal', 'current', 'chip', 'analog', 'bit', 'voltage', 'channel', 'implementation', 'source', 'design', 'neural', 'noise', 'processor', 'computation', 'memory', 'code', 'parallel', 'operation', 'application']

Topic #2:
['class', 'training', 'word', 'classification', 'classifier', 'recognition', 'feature', 'pattern', 'speech', 'trained', 'sequence', 'character', 'context', 'experiment', 'hmm', 'vector', 'probability', 'test', 'frame', 'mlp']

Topic #3:
['distribution', 'gaussian', 'variable', 'probability', 'estimate', 'matrix', 'prior', 'equation', 'component', 'density', 'vector', 'noise', 'mixture', 'approximation', 'variance', 'estimation', 'bayesian', 'eq', 'sample', 'log']

Topic #4:
['image', 'feature', 'object', 'representation', 'region', 'pixel', 'local', 'distance', 'cluster', 'view', 'face', 'surface', 'structure', 'location', 'clustering', 'vector', 'map', 'shape', 'transformation', 'position']

Topic #5:
['unit', 'layer', 'node', 'pattern', 'rule',

We can also evaluate our model using the perplexity and coherence metrics, as we
did before.

In [0]:
cv_coherence_model_lda_mallet = gensim.models. CoherenceModel (model=lda_mallet, corpus=bow_corpus,
                                                               texts=norm_corpus_bigrams, dictionary=dictionary,
                                                               coherence='c_v')

avg_coherence_cv = cv_coherence_model_lda_mallet.get_coherence()

umass_coherence_model_lda_mallet = gensim.models. CoherenceModel (model=lda_mallet, corpus=bow_corpus,
                                                                   texts=norm_corpus_bigrams, dictionary=dictionary,
                                                                  coherence='u_mass') 

avg_coherence_umass = umass_coherence_model_lda_mallet.get_coherence()

In [81]:
# from STDOUT: <500> LL/token: -8.53533
perplexity = -8.53533

print('Avg. Coherence Score (Cv):', avg_coherence_cv)
print('Avg. Coherence Score (UMass):', avg_coherence_umass)
print('Model Perplexity:', perplexity)

Avg. Coherence Score (Cv): 0.5143469304857831
Avg. Coherence Score (UMass): -1.038940764502137
Model Perplexity: -8.53533


You can clearly see that the model from MALLET is much better based on these
metrics as compared to the default LDA model from Gensim. Can we find the optimal number of topics that maximizes the coherence? This is a tough problem, but we can try doing it iteratively.